In [2]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import emotion_model
import dwdii_transforms

random.seed(20275)

Using Theano backend.


In [3]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [4]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"
imgResize = (150, 150)

In [5]:
os.listdir('/root/facial_expressions/data')

['legend.csv', '500_picts_satz.csv']

### Load Training and Test Data

In [6]:
#emoMetaData = dwdii_transforms.load_training_metadata(dataPath, True)

In [7]:
#len(emoMetaData)

In [8]:
maxData = 1823
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 200, imgResize=imgResize)
print X_data.shape
print Y_data.shape

0.000000: King_Abdullah_II_0001.jpg
0.109709: Lleyton_Hewitt_0041.jpg
0.219419: Dwarakish_30.jpg
0.219419: Megawati_Sukarnoputri_0018.jpg
0.329128: HrithikRoshan_174.jpg
0.329128: Valerie_Thwaites_0001.jpg
0.438837: HrithikRoshan_115.jpg
0.438837: Dwarakish_180.jpg
0.438837: HrithikRoshan_3.jpg
0.438837: George_W_Bush_0422.jpg
0.548546: Serena_Williams_0015.jpg
0.658256: Bob_Riley_0001.jpg
0.767965: Lindsay_Davenport_0012.jpg
0.877674: HrithikRoshan_112.jpg
0.877674: Sergio_Vieira_De_Mello_0002.jpg
0.987383: Trent_Lott_0016.jpg
(1817, 150, 150)
(1817, 1)


### Transformations
In this section, we will apply transformations to the existing images to increase of training data, as well as add a bit of noise in the hopes of improving the overall training activities.

In [11]:
newImgs = np.zeros([X_data.shape[0] * 9, X_data.shape[1], X_data.shape[2]])
newYs = np.zeros([Y_data.shape[0] * 9, Y_data.shape[1]], dtype=np.int8)
print newImgs.shape
print newYs.shape

(16353, 150, 150)
(16353, 1)


In [12]:
ndx = 0
for i in range(X_data.shape[0]):
    img = X_data[i]
    
    img0 = dwdii_transforms.reflectY(img)
    newImgs[ndx] = img0
    newYs[ndx] = Y_data[i]
    #misc.imsave("test0.png", img0)
    ndx += 1
    
    img1 = dwdii_transforms.cvDilate(img)
    newImgs[ndx] = img1
    newYs[ndx] = Y_data[i]
    #misc.imsave("test1.png", img1)
    ndx += 1
    
    img2 = dwdii_transforms.cvErode(img)
    newImgs[ndx] = img2
    newYs[ndx] = Y_data[i]
    #misc.imsave("test2.png", img2)
    ndx += 1
    
    img3 = dwdii_transforms.cvDilate2(img)
    newImgs[ndx] = img3
    newYs[ndx] = Y_data[i]
    #misc.imsave("test3.png", img3)
    ndx += 1    
    
    #img4 = dwdii_transforms.cvMedianBlur(img)
    #newImgs[ndx] = img4
    #newYs[ndx] = Y_data[i]
    #misc.imsave("test4.png", img4)
    #ndx += 1      
    
    img5 = dwdii_transforms.cvExcessiveSharpening(img)
    newImgs[ndx] = img5
    newYs[ndx] = Y_data[i]
    #misc.imsave("test5.png", img5)
    ndx += 1    
    
    img6 = dwdii_transforms.cvEdgeEnhancement(img)
    newImgs[ndx] = img6
    newYs[ndx] = Y_data[i]
    #misc.imsave("test6.png", img6)
    ndx += 1    

    img7 = dwdii_transforms.cvBlurMotion1(img)
    newImgs[ndx] = img7
    newYs[ndx] = Y_data[i]
    #misc.imsave("test7.png", img7)
    ndx += 1    
    
    img8 = dwdii_transforms.cvBlurMotion2(img)
    newImgs[ndx] = img8
    newYs[ndx] = Y_data[i]
    #misc.imsave("test8.png", img8)
    ndx += 1      

    img9 = dwdii_transforms.reflectY(img)
    newImgs[ndx] = img9
    newYs[ndx] = Y_data[i]
    #misc.imsave("test9.png", img9)
    ndx += 1      

    #break
    
print("Done", str(datetime.datetime.now()))

('Done', '2016-12-11 19:57:27.021505')


In [13]:
gc.collect()

9

In [14]:
X_data2 = np.concatenate((X_data, newImgs))
Y_data2 = np.concatenate((Y_data, newYs))
print X_data2.shape
print Y_data2.shape

(18170, 150, 150)
(18170, 1)


### Split Training/Test Sets

The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the `unison_shuffled_copies` help method.

In [15]:
skippedTransforms = False
if skippedTransforms:
    X_data2 = X_data
    Y_data2 = Y_data

In [16]:
def unison_shuffled_copies(a, b):
    """http://stackoverflow.com/a/4602224/2604144"""
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

# First shuffle the data 
X_data2, Y_data2 = unison_shuffled_copies(X_data2, Y_data2)

# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx
X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

14536
(14536, 150, 150)
(3634, 150, 150)
(14536, 1)
(3634, 1)


### Define the Model
In this section, we define the model. The `emotion_model` module contains the model definition itself. `emotion_model_v1` is a basic convolutional neural network while our final model is a variation on a model shared by James Hamski (jh).

The model is trained on 1 of 8 emotions.

In [17]:
# Map the emotions to integers for categorization later.
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

{'sadness': 6, 'neutral': 2, 'contempt': 7, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
8


Our model is a convolutional neural network with 4 hidden layers.  

In [18]:
#model = emotion_model.emotion_model_v3_2(len(emotions), verbose=True)
model = emotion_model.emotion_model_jh_v5(len(emotions), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))
#print(model.summary())

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 143, 143)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

### Training the Model

The following code segment trains the model using the `run_network` helper function. Previously, I was hitting a memory issue (my interpretation), when I have batches above a certain threshold. Batches=10 work fine, but batches of 100 are too big. May need to allocate more RAM to the docker container. I have since moved to a Docker Cloud / Amazon Web Services instance with increased memory and this issue has been mitigated.

In [19]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_train.shape[1],X_train.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [20]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-emo-01vjh-1-Cloud.hdf5")

In [21]:
print "Training start: " + str(datetime.datetime.now())
m, h = emotion_model.run_network([trainX, testX, Y_train, Y_test], model, batch=200, epochs=100, verbosity=1)

Training start: 2016-12-11 20:04:39.495916
(14536, 8)
(3634, 8)
Training model...
Train on 14536 samples, validate on 3634 samples
Epoch 1/100
14536/14536 [==============================] - 952s - loss: 0.3321 - acc: 0.3789 - val_loss: 0.2828 - val_acc: 0.5146
Epoch 2/100
14536/14536 [==============================] - 956s - loss: 0.2485 - acc: 0.5474 - val_loss: 0.2280 - val_acc: 0.5806
Epoch 3/100
14536/14536 [==============================] - 957s - loss: 0.2108 - acc: 0.6278 - val_loss: 0.2225 - val_acc: 0.5897
Epoch 4/100
14536/14536 [==============================] - 961s - loss: 0.1835 - acc: 0.6835 - val_loss: 0.1784 - val_acc: 0.6973
Epoch 5/100
14536/14536 [==============================] - 959s - loss: 0.1535 - acc: 0.7405 - val_loss: 0.1616 - val_acc: 0.7320
Epoch 6/100
14536/14536 [==============================] - 961s - loss: 0.1282 - acc: 0.7915 - val_loss: 0.1268 - val_acc: 0.7917
Epoch 7/100
14536/14536 [==============================] - 961s - loss: 0.1053 - acc: 0.8

* Model v2: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 4474s 
  * #2: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 5469s 
* Model v3: Network's test score [loss, accuracy]: [8.0187525367736825, 0.50249999999999995] - 2499s
* Model v4.1: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 2137s 
* Movel v4.2: Network's test score [loss, accuracy]: [8.0187525367736825, 0.50249999999999995] - 3108s 
* Model v5: Network's test score [loss, accuracy]: [6.9666682052612305, 0.19500000000000001] - 1682s 
* Model v6: Network's test score [loss, accuracy]: [1.7120025205612182, 0.23749999999999999] - 3020s 
* Model v7: Network's test score [loss, accuracy]: [7.9999716758728026, 0.14000000000000001] - 4610s 
* Model v8: Network's test score [loss, accuracy]: [1.6948303937911988, 0.19500000000000001] - 3313s 
* Model v6 w/ flatten: Network's test score [loss, accuracy]: [7.1107604598999021, 0.17249999999999999] - 3044s 
* Model v6 (Docker Cloud): Network's test score [loss, accuracy]: [11.153776299942534, 0.307974335472044] - 3597s 
* Model v3.1: Network's test score [loss, accuracy]: [1.5994336946608279, 0.22314049603196873] - 687s 
* Model v3.2: Network's test score [loss, accuracy]: [1.5697537031802502, 0.35989011021760792] - 772s
* cnn_model_jhamski: Network's test score [loss, accuracy]: [0.31684105933367551, 0.34615384648134423] - 636s
 * Many epochs 150x150: Network's test score [loss, accuracy]: [0.51708218340690315, 0.6428571428571429]
* emotion_model_jh_v2: Network's test score [loss, accuracy]: [0.27507745529690836, 0.55616438421484538] - 2027s
  * Epoch x20: Network's test score [loss, accuracy]: [0.32478914950808435, 0.63287671265536793]
* v3 epoch x40: Network's test score [loss, accuracy]: [0.078623215722688183, 0.88359303391384048]
  * +20: Network's test score [loss, accuracy]: [0.070701496646681364, 0.91750687442713108]
* v4 Epoch 20: Network's test score [loss, accuracy]: [0.23540275704827893, 0.61279229702942961]
  * +80: Network's test score [loss, accuracy]: [0.1823677838099789, 0.8466299859988804]
* v5 Epoch 20: Network's test score [loss, accuracy]: [0.13165531713295181, 0.78610729039781191]
  * +80: Network's test score [loss, accuracy]: [0.097839370133615211, 0.9312242091603915]
* v5 Epoch 27 - 13082 examples (8 transforms): val_loss: 0.1024 - val_acc: 0.9242
* v5 Epoch 27 - 14536 examples (9 transforms): val_loss: 0.0941 - val_acc: 0.9265

In [23]:
model.save_weights("dwdii-emo-150-jhv5-9tf-Cloud.hdf5", overwrite=True)

### Precision & Recall

In this section we compute Precision and Recall metrics for each of the emotion classes.

In [24]:
predictOutput = model.predict(testX)

In [25]:
predictOutput[0]

array([  1.46960699e-09,   4.32014586e-14,   9.98015821e-01,
         5.17242484e-07,   5.25253341e-10,   1.04667470e-06,
         1.73934808e-07,   1.98241440e-03])

In [46]:
import collections
prMetrics = {}

# For each emotion
for e in emotions.keys():
    prMetrics[e] = collections.defaultdict(int)   
print prMetrics

numEmo = dwdii_transforms.numericEmotions()
print numEmo

{'neutral': defaultdict(<type 'int'>, {}), 'sadness': defaultdict(<type 'int'>, {}), 'happiness': defaultdict(<type 'int'>, {}), 'disgust': defaultdict(<type 'int'>, {}), 'anger': defaultdict(<type 'int'>, {}), 'surprise': defaultdict(<type 'int'>, {}), 'fear': defaultdict(<type 'int'>, {}), 'contempt': defaultdict(<type 'int'>, {})}
{0: 'anger', 1: 'disgust', 2: 'neutral', 3: 'happiness', 4: 'surprise', 5: 'fear', 6: 'sadness', 7: 'contempt'}


In [47]:
# For each predicted image
for i in range(len(predictOutput)):

    arPred = np.array(predictOutput[i])
    predictionProb = arPred.max()
    predictionNdx = arPred.argmax()
    predictedEmo = numEmo[predictionNdx]

    # True Positives
    if predictionNdx == Y_test[i]:
        prMetrics[predictedEmo]["TruePos"] += 1.0
    # False Positives
    else:
        prMetrics[predictedEmo]["FalsePos"] += 1.0
        
# Look for false negatives
for i in range(len(Y_test)):
    
    arPred = np.array(predictOutput[i])
    predictionProb = arPred.max()
    predictionNdx = arPred.argmax()
    predictedEmo = numEmo[predictionNdx]
    
    
    yEmo = numEmo[int(Y_test[i])]
    if Y_test[i] == predictionNdx:
        # Ok
        pass
    else:
        prMetrics[yEmo]["FalseNeg"] += 1.0

In [48]:
prMetrics

{'anger': defaultdict(int,
             {'FalseNeg': 80.0, 'FalsePos': 56.0, 'TruePos': 384.0}),
 'contempt': defaultdict(int,
             {'FalseNeg': 6.0, 'FalsePos': 8.0, 'TruePos': 13.0}),
 'disgust': defaultdict(int,
             {'FalseNeg': 10.0, 'FalsePos': 1.0, 'TruePos': 19.0}),
 'fear': defaultdict(int, {'FalseNeg': 6.0, 'FalsePos': 1.0, 'TruePos': 18.0}),
 'happiness': defaultdict(int,
             {'FalseNeg': 223.0, 'FalsePos': 10.0, 'TruePos': 802.0}),
 'neutral': defaultdict(int,
             {'FalseNeg': 177.0, 'FalsePos': 105.0, 'TruePos': 936.0}),
 'sadness': defaultdict(int,
             {'FalseNeg': 39.0, 'FalsePos': 54.0, 'TruePos': 222.0}),
 'surprise': defaultdict(int,
             {'FalseNeg': 13.0, 'FalsePos': 319.0, 'TruePos': 686.0})}

In [49]:
emotionPrecision = {}
emotionRecall = {}
for p in prMetrics:
    emotionPrecision[p] = prMetrics[p]["TruePos"] / ( prMetrics[p]["TruePos"] + prMetrics[p]["FalsePos"])
    emotionRecall[p] = prMetrics[p]["TruePos"] /( prMetrics[p]["TruePos"] + prMetrics[p]["FalseNeg"])
    
print "Precision by Emotion"
print "--------------------"
for e in emotionPrecision:
    print e, emotionPrecision[e]
print
print "Recall by Emotion"
print "--------------------"
for e in emotionRecall:
    print e, emotionRecall[e]    

Precision by Emotion
--------------------
sadness 0.804347826087
neutral 0.899135446686
contempt 0.619047619048
disgust 0.95
anger 0.872727272727
surprise 0.682587064677
fear 0.947368421053
happiness 0.987684729064

Recall by Emotion
--------------------
sadness 0.850574712644
neutral 0.840970350404
contempt 0.684210526316
disgust 0.655172413793
anger 0.827586206897
surprise 0.981402002861
fear 0.75
happiness 0.78243902439


### References

* OpenCV/CV2: http://askubuntu.com/questions/447409/how-to-install-opencv-2-9-for-python
* Docker Commit: http://stackoverflow.com/questions/19585028/i-lose-my-data-when-the-container-exits
  * docker ps -l
  * docker commit <ContainerID> dittenhafer/dl
* http://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/